In [12]:
import numpy as np
import pandas as pd
import os

map_name = 'HRL3'

# Load MPC data
mpc_csv_file_name = 'mpc_data_1_4.csv'
mpc_csv_file_path = os.path.join(map_name, mpc_csv_file_name)
mpc_csv = pd.read_csv(mpc_csv_file_path)
print(mpc_csv)


      Iteration       s_cur     w_cur    mu_cur         v         angle  \
0             2   74.217653  0.335779 -0.752138  2.000000  0.000000e+00   
1             3   74.233920  0.294345 -0.752138  1.500000 -3.000000e-01   
2             4   74.283083  0.249805 -0.664348  1.500000  1.214921e-14   
3             5   74.316007  0.204357 -0.674317  1.500000  3.000000e-01   
4             6   74.374406  0.158889 -0.615090  1.500000  4.438701e-01   
...         ...         ...       ...       ...       ...           ...   
1494       1496  121.127939 -0.011893 -0.005647  2.792982  2.007215e-02   
1495       1497  121.261184 -0.012330 -0.005487  2.806648  1.768389e-02   
1496       1498  121.403033 -0.013668 -0.011613  2.819335  2.985897e-02   
1497       1499  121.534758 -0.014316 -0.011966  2.832333  2.629047e-02   
1498       1500  121.675266 -0.015975 -0.019116  2.844820  4.132759e-02   

      Racecar X  Racecar Y   Racecar heading  
0     31.499923  56.500000          0.000000  
1    

In [13]:
# Transfer mpc x,y,vx to numpy array
mpc_data = mpc_csv[['Racecar X', 'Racecar Y', 'v', 'angle']].to_numpy()
print(mpc_data)

[[ 3.14999226e+01  5.65000000e+01  2.00000000e+00  0.00000000e+00]
 [ 3.15595962e+01  5.65000000e+01  1.50000000e+00 -3.00000000e-01]
 [ 3.16275181e+01  5.65000000e+01  1.50000000e+00  1.21492098e-14]
 ...
 [ 7.50473688e+01  4.95598700e+01  2.81933520e+00  2.98589680e-02]
 [ 7.50758162e+01  4.94248692e+01  2.83233316e+00  2.62904666e-02]
 [ 7.51052515e+01  4.92893913e+01  2.84481966e+00  4.13275930e-02]]


In [14]:
# Load centerline
centerline_file_name = f'{map_name}_centerline.csv'
centerline_file_path = os.path.join(map_name, centerline_file_name)
centerline_csv = pd.read_csv(centerline_file_path)

# Transfer centerline to numpy
centerline_data = centerline_csv[['x','y']].to_numpy()
print(centerline_data)

[[1449.  199.]
 [1429.  205.]
 [1409.  216.]
 [1389.  226.]
 [1369.  234.]
 [1349.  243.]
 [1329.  251.]
 [1309.  260.]
 [1289.  268.]
 [1269.  277.]
 [1249.  286.]
 [1209.  302.]
 [1189.  311.]
 [1169.  320.]
 [1149.  328.]
 [1129.  337.]
 [1109.  345.]
 [1089.  354.]
 [1069.  362.]
 [1049.  371.]
 [1009.  387.]
 [ 989.  396.]
 [ 929.  420.]
 [ 909.  429.]
 [ 889.  437.]
 [ 869.  446.]
 [ 849.  454.]
 [ 829.  462.]
 [ 809.  469.]
 [ 789.  477.]
 [ 769.  484.]
 [ 749.  492.]
 [ 729.  499.]
 [ 669.  523.]
 [ 629.  541.]
 [ 611.  558.]
 [ 608.  578.]
 [ 609.  598.]
 [ 609.  618.]
 [ 611.  638.]
 [ 612.  658.]
 [ 614.  678.]
 [ 616.  718.]
 [ 618.  738.]
 [ 622.  818.]
 [ 623.  838.]
 [ 623.  858.]
 [ 624.  878.]
 [ 624.  898.]
 [ 626.  938.]
 [ 627.  958.]
 [ 627.  978.]
 [ 628.  998.]
 [ 629. 1018.]
 [ 629. 1038.]
 [ 630. 1058.]
 [ 630. 1078.]
 [ 628. 1098.]
 [ 629. 1118.]
 [ 642. 1131.]
 [ 662. 1132.]
 [ 682. 1131.]
 [ 702. 1129.]
 [ 722. 1126.]
 [ 742. 1124.]
 [ 762. 1124.]
 [ 782. 11

In [15]:
# Load map resolution
map_config_path = os.path.join(map_name, f'{map_name}_map.yaml')

import yaml
with open(map_config_path, 'r') as file:
    map_config_dict = yaml.safe_load(file)
print(map_config_dict)

map_resolution = map_config_dict['resolution']

{'image': 'HRL3_map.png', 'negate': 0, 'occupied_thresh': 0.75, 'free_thresh': 0.25, 'resolution': 0.05, 'origin': [0.0, 0.0, 0.0], 'starting_pose': [31.4, 56.5, 0.0], 'starting_index': 8}


In [16]:
# Transfer centerline data into resolution
centerline_data = centerline_data*map_resolution

In [17]:
# Cut the start and end of mpc data 
print(mpc_data[0], mpc_data.shape[0])
mpc_data_clipped = mpc_data[25:-25]
print(mpc_data_clipped[0], mpc_data_clipped.shape[0])

[31.4999226 56.5        2.         0.       ] 1499
[33.42327733 56.67524129  1.96987792 -0.06128413] 1449


In [18]:
# Find nearest point in centerline for each point in mpc data and add velocity 
centerline_data_vx = np.zeros(shape=(centerline_data.shape[0],4))

mpc_data_xy = mpc_data_clipped[:, :2]

# Finde für jeden Punkt in array1 den Index des nächsten Punktes in array2
nearest_indices = []

for point in centerline_data:
    # Berechne die quadratische Distanz zu allen Punkten in array2
    distances = np.sum((mpc_data_xy - point) ** 2, axis=1)
    # Finde den Index des minimalen Werts (nächster Punkt)
    nearest_index = np.argmin(distances)
    nearest_indices.append(nearest_index)

# Ausgabe der Indizes der nächstgelegenen Punkte
print(nearest_indices)

for i in range(centerline_data_vx.shape[0]):
    centerline_data_vx[i] = np.array([centerline_data[i][0], centerline_data[i][1], mpc_data_clipped[nearest_indices[i]][2], mpc_data_clipped[nearest_indices[i]][3]])
    print(centerline_data_vx[i])

[np.int64(594), np.int64(605), np.int64(615), np.int64(625), np.int64(633), np.int64(641), np.int64(649), np.int64(657), np.int64(664), np.int64(671), np.int64(678), np.int64(692), np.int64(698), np.int64(705), np.int64(711), np.int64(718), np.int64(724), np.int64(730), np.int64(737), np.int64(743), np.int64(755), np.int64(761), np.int64(779), np.int64(785), np.int64(792), np.int64(798), np.int64(804), np.int64(811), np.int64(818), np.int64(825), np.int64(832), np.int64(840), np.int64(848), np.int64(871), np.int64(886), np.int64(896), np.int64(905), np.int64(914), np.int64(922), np.int64(929), np.int64(937), np.int64(944), np.int64(957), np.int64(964), np.int64(989), np.int64(995), np.int64(1001), np.int64(1008), np.int64(1014), np.int64(1028), np.int64(1036), np.int64(1044), np.int64(1054), np.int64(1064), np.int64(1077), np.int64(1091), np.int64(1104), np.int64(1117), np.int64(1130), np.int64(1144), np.int64(1157), np.int64(7), np.int64(1178), np.int64(25), np.int64(1195), np.int64(4

In [19]:
# save as csv 
output_file_name = f'{map_name}_centerline_mpc.csv'
centerline_mpc = pd.DataFrame(centerline_data_vx, columns=['x', 'y', 'v', 'r'])
centerline_mpc.to_csv(os.path.join(map_name, output_file_name), index=False)